In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical


In [ ]:
labels = { 0: "Speed limit (20km/h)", 
           1: "Speed limit (30km/h)", 
           2: "Speed limit (50km/h)", 
           3: "Speed limit (60km/h)", 
           4: "Speed limit (70km/h)", 
           5: "Speed limit (80km/h)",
           6: "End of Speed limit (80km/h)", 
           7: "Speed limit (100km/h)", 
           8: "Speed limit (120km/h)",
           9: "No passing", 
           10: "No passing veh over 3.5 tons", 
           11: "Right-of-way at intersection",
           12: "Priority road", 
           13: "Yield", 
           14: "Stop", 
           15: "No vehicles", 
           16: "Veh > 3.5 tons prohibited",
           17: "No entry", 
           18: "General caution", 
           19: "Dangerous curve left", 
           20: "Dangerous curve right", 
           21: "Double curve", 
           22: "Bumpy road", 
           23: "Slippery road", 
           24: "Road narrows on the right",
           25: "Road work", 
           26: "Traffic signals", 
           27: "Pedestrians", 
           28: "Children crossing", 
           29: "Bicycle crossing",
           30: "Beware of ice/snow", 
           31: "Wild animals crossing", 
           32: "End speed + passing limits", 
           33: "Turn right ahead", 
           34: "Turn left ahead", 
           35: "Ahead only", 
           36: "Go straight or right", 
           37: "Go straight or left", 
           38: "Keep right", 
           39: "Keep Left", 
           40: "Roundabout mandatory",
           41: "End of no passing", 
           42: "End of no passing veh > 3.5 tons"}

In [ ]:
train = pd.read_csv('/kaggle/input/gtsrb-german-traffic-sign/Train.csv')


In [ ]:
train.shape

In [ ]:
train_labels=train['ClassId'].unique()
train_labels

In [ ]:
train.reset_index()

In [ ]:
df=train.iloc[:,6:]
df

In [ ]:
df['Path']= '/kaggle/input/gtsrb-german-traffic-sign/' + train['Path']
df


In [ ]:
df['Path']

In [ ]:
plt.imshow(cv2.imread(df["Path"][5]))
print(labels[(df['ClassId'][5])])

In [ ]:
test = pd.read_csv('/kaggle/input/gtsrb-german-traffic-sign/Test.csv')
test['Path']= '/kaggle/input/gtsrb-german-traffic-sign/' + test['Path']
test=test.iloc[:,6:]
test

In [ ]:
training_data = []

for num in range(len(df)):
    img = cv2.imread(df['Path'][num])
#     converting to gray color i.e 1D 
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img.astype('uint8'))
#   normalize to the range of 0 and 1 (normalization of data)
    img = img/255.00
    img = cv2.resize(img, (33,33),  
               interpolation = cv2.INTER_NEAREST)
    training_data.append(img)
    
training_data = np.array(training_data)

In [ ]:
testing_data = []
#print(training_images_path)
for num in range(len(test)):
    img = cv2.imread(test['Path'][num])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img.astype('uint8'))

    img = img/255.00
    img = cv2.resize(img, (33,33),  
               interpolation = cv2.INTER_NEAREST)
    testing_data.append(img)
    
testing_data = np.array(testing_data)



In [ ]:
print(training_data.shape)
print(testing_data.shape)

In [ ]:
training_data = training_data.reshape(training_data.shape[0],training_data.shape[1],training_data.shape[2],1)

testing_data = testing_data.reshape(testing_data.shape[0],testing_data.shape[1],testing_data.shape[2],1)

In [ ]:
print(training_data.shape)
print(testing_data.shape)

# Data Augmentation

Training data generation for better results. Images are rotated and zoomed for taking all possibilities into consideration.

In [ ]:
datagen = ImageDataGenerator(width_shift_range = 0.1,
                            height_shift_range = 0.1,
                            zoom_range = 0.2,
                            shear_range = 0.1,
                            rotation_range = 10,
                            horizontal_flip=False)


In [ ]:
datagen.fit(training_data)

In [ ]:
y_train = to_categorical(df.ClassId)
y_test= to_categorical(test.ClassId)

In [ ]:
batch = datagen.flow(training_data,y_train,batch_size = 20)
i,l=next(batch)
plt.imshow(i[7].reshape(33,33))
# catoegorical encoding :- 1 will be assign to label
print(l[7])

print(np.argmax(l[7]))

# Convolutional Neural Network

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping

# Early stopping criterion
stop = EarlyStopping(monitor  = 'val_accuracy', patience = 5,restore_best_weights=True) # early stopping callbacks

model = Sequential()

# relu is used for continuous values
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', kernel_initializer='he_uniform',input_shape=(33,33,1)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer='he_uniform'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())

model.add(Dense(256, activation='relu',kernel_initializer='he_uniform'))
model.add(Dropout(rate=0.5))

#for categorical
model.add(Dense(43, activation='softmax'))

model.summary()


In [ ]:
#Compilation of the model
#categorical crossentropy for more than categories, 0-42
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [ ]:
pred = model.predict(testing_data)
result = np.argmax(pred, axis = 1)
from sklearn.metrics import accuracy_score
accuracy_score(test.ClassId,result)

In [ ]:
flow=datagen.flow(training_data,y_train, batch_size = 20)
steps=int(len(df)/20)
history=model.fit_generator(flow , steps_per_epoch=steps , epochs=32 , validation_data= (testing_data,y_test), verbose=1,callbacks=[stop])

In [ ]:
model.save('/kaggle/working/model_traffic_data.h5')

In [ ]:
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()